In [ ]:
%run notebook_setup

In [ ]:
run_trace = True # set to True to rerun all traces in full; otherwise loads saved
import pandas as pd
import corner
import astropy.units as u

### Load up HARPS data & fit

In [ ]:
df = pd.read_csv("../../data/harps_rvs.csv")
for k in df.columns:
    new_k = k.strip("#").strip()
    if new_k != k:
        df[new_k] = df[k]
        del df[k]

In [ ]:
BTJD_ref = 2457000
HARPS_upgrade = 2457218.5 # July 2015

# Remove one bisector outlier
df = df[df.bis < df.bis.max()]
        
df = df.sort_values("date")

pug = np.ascontiguousarray(df.bjd >= HARPS_upgrade, dtype=bool) # post-upgrade mask

t_h1 = np.ascontiguousarray(df.bjd[~pug], dtype=np.float64)
rv_h1 = np.ascontiguousarray(df.rv[~pug], dtype=np.float64)
rverr_h1 = np.ascontiguousarray(df.e_rv[~pug], dtype=np.float64)
#fwhm_h1 = np.ascontiguousarray((df.fwhm[~pug] - df.fwhm.mean()) / df.fwhm.std(), dtype=np.float64)
fwhm_h1 = np.ascontiguousarray(df.fwhm[~pug], dtype=np.float64)
fwhm_h1 -= np.median(fwhm_h1)
rv_h1 -= np.median(rv_h1)

t_h2 = np.ascontiguousarray(df.bjd[pug], dtype=np.float64)
rv_h2 = np.ascontiguousarray(df.rv[pug], dtype=np.float64)
rverr_h2 = np.ascontiguousarray(df.e_rv[pug], dtype=np.float64)
#fwhm_h2 = np.ascontiguousarray((df.fwhm[pug] - df.fwhm.mean()) / df.fwhm.std(), dtype=np.float64)
fwhm_h2 = np.ascontiguousarray(df.fwhm[pug], dtype=np.float64)
fwhm_h2 -= np.median(fwhm_h2)
rv_h2 -= np.median(rv_h2)

#### fixed zero eccentricity:

In [ ]:
rv_trend_order = 1

t_dense = np.linspace(t_h1.min()-5, t_h2.max()+5, 1000)

with pm.Model() as model:
    # Keplerian parameters
    period = pm.Normal("period", mu=17.47128, sd=0.00005)
    t0 = pm.Normal("t0", mu=2458661.0628, sd=0.0007)   
    K = pm.Uniform("K", lower=0., upper=10., testval=5.)
    ecc = 0.
    omega = np.pi
    
    # RV offsets
    delta_rv = pm.Uniform("delta_rv", lower=-10., upper=10.) # mu_h2 - trend[1]
    if rv_trend_order == 1:
        trend = pm.Uniform("trend", lower=-10., upper=10.) # ideally this would be shape [1] but idk how to make that work so terrible hacks will follow
        #trend = pm.Normal("trend", mu=0, sd=3)
    else:
        trend = pm.Normal("trend", mu=0, sd=10.0**(1-np.arange(rv_trend_order))[::-1], shape=rv_trend_order)
    
    # RV jitter
    logs_h1 = pm.Uniform("logs_h1", lower=0., upper=10.)
    logs_h2 = pm.Uniform("logs_h2", lower=0., upper=10.)
    
    # FWHM trend
    delta_fwhm = pm.Normal("delta_fwhm", mu=1., sd=5) # weak prior from eyeballing
    trend_fwhm = pm.Normal("trend_fwhm", mu=0, sd=10.0**(1-np.arange(2))[::-1], shape=2)
            
    # Orbit model
    orbit_rvs = xo.orbits.KeplerianOrbit(
        period=period,
        ecc=ecc, omega=omega)
    
    # Save some helpful things for later
    semimajor = orbit_rvs.a
    pm.Deterministic('a', semimajor)
    
    # Set up the RV model and save it as a deterministic
    # for plotting purposes later
    vrad_h1 = orbit_rvs.get_radial_velocity(t_h1, K=K)
    vrad_h2 = orbit_rvs.get_radial_velocity(t_h2, K=K)
    vrad = orbit_rvs.get_radial_velocity(np.append(t_h1, t_h2), K=K)
    pm.Deterministic("vrad", vrad)
    
    # RV noise model for h1
    A_fwhm_h1 = np.vander(fwhm_h1, 2)
    bkg_terms_h1 = tt.dot(A_fwhm_h1, trend_fwhm)
    if rv_trend_order == 1:
        bkg_terms_h1 += trend
    else:
        A_trend_h1 = np.vander(t_h1, rv_trend_order)
        bkg_terms_h1 += tt.dot(A_trend_h1, trend)
    bkg_h1 = pm.Deterministic("bkg_h1", bkg_terms_h1)
        
    # RV noise model for h2
    A_fwhm_h2 = np.vander(fwhm_h2, 2)
    bkg_terms_h2 = tt.dot(A_fwhm_h2, trend_fwhm) - delta_fwhm*trend_fwhm[0]
    if rv_trend_order == 1:
        bkg_terms_h2 += trend + delta_rv
    else:
        A_trend_h2 = np.vander(t_h2, rv_trend_order)
        bkg_terms_h2 += tt.dot(A_trend_h2, trend) + delta_rv
    bkg_h2 = pm.Deterministic("bkg_h2", bkg_terms_h2)

    # The likelihood for the RVs
    rv_model_h1 = pm.Deterministic("rv_model_h1", vrad_h1 + bkg_h1)  
    rv_model_h2 = pm.Deterministic("rv_model_h2", vrad_h2 + bkg_h2)        
    err_h1 = tt.sqrt(rverr_h1**2 + tt.exp(2*logs_h1))
    err_h2 = tt.sqrt(rverr_h2**2 + tt.exp(2*logs_h2))
    pm.Normal("obs_h1", mu=rv_model_h1, sd=err_h1, observed=rv_h1)
    pm.Normal("obs_h2", mu=rv_model_h2, sd=err_h2, observed=rv_h2)
        
    vrad_pred = orbit_rvs.get_radial_velocity(t_dense, K=K)
    pm.Deterministic("vrad_pred", vrad_pred)
    A_pred = np.vander(t_dense, rv_trend_order)        
    bkg_pred = pm.Deterministic("bkg_pred", tt.dot(A_pred, trend))
    pm.Deterministic("rv_model_pred", vrad_pred + bkg_pred)

    # Fit for the maximum a posteriori parameters
    start = model.test_point
    map_soln = start
    map_soln = xo.optimize(start=map_soln, vars=[logs_h1, logs_h2, trend, delta_rv])
    map_soln = xo.optimize(start=map_soln, vars=[trend_fwhm, delta_fwhm])
    map_soln = xo.optimize(start=map_soln, vars=[period, t0])
    map_soln = xo.optimize(start=map_soln, vars=[K])
    map_soln = xo.optimize(start=map_soln)

In [ ]:
if run_trace:
    with model:
        trace = pm.sample(tune=1000, draws=1000, start=map_soln, chains=2,
                      step=xo.get_dense_nuts_step(target_accept=0.9))
    pm.save_trace(trace,directory='trace_fwhm_noecc_o{0}'.format(rv_trend_order), overwrite=True)   
else:
    with model:
        trace = pm.load_trace('trace_fwhm_noecc_o{0}'.format(rv_trend_order))

In [ ]:
for varname in ["period", "t0", "K", "delta_rv", "delta_fwhm", 
                "trend", "trend_fwhm", "logs_h1", "logs_h2"]:
    print("{0}: median {1} +{2} -{3}; MAP {4}".format(varname, np.median(trace[varname], axis=0), 
                                                      np.percentile(trace[varname], [84], axis=0) - np.median(trace[varname], axis=0), 
                                                      np.median(trace[varname], axis=0) - np.percentile(trace[varname], [16], axis=0), 
                                                      map_soln[varname]))

In [ ]:
print('period: {0:.7f} + {1:.7f} - {2:.7f}'.format(np.median(trace['period']), 
                                                    np.percentile(trace['period'], 84) - np.median(trace['period']), 
                                                    np.median(trace['period']) - np.percentile(trace['period'], 16)))
print('t0: {0:.5f} + {1:.5f} - {2:.5f}'.format(np.median(trace['t0']), 
                                                    np.percentile(trace['t0'], 84) - np.median(trace['t0']), 
                                                    np.median(trace['t0']) - np.percentile(trace['t0'], 16)))
print('K: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.median(trace['K']), 
                                                    np.percentile(trace['K'], 84) - np.median(trace['K']), 
                                                    np.median(trace['K']) - np.percentile(trace['K'], 16)))
if rv_trend_order>1:
    print('mu_h1: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.median(trace['trend'], axis=0)[-1], 
                                                    np.percentile(trace['trend'], 84, axis=0)[-1] - np.median(trace['trend'], axis=0)[-1], 
                                                    np.median(trace['trend'], axis=0)[-1] - np.percentile(trace['trend'], 16, axis=0)[-1]))
    print('mu_h2: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.median(trace['trend'], axis=0)[-1] + np.median(trace['delta_rv']), 
                                                    np.percentile(trace['delta_rv'], 84) - np.median(trace['delta_rv']), 
                                                    np.median(trace['delta_rv']) - np.percentile(trace['delta_rv'], 16)))
else:
    print('mu_h1: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.median(trace['trend']), 
                                                    np.percentile(trace['trend'], 84) - np.median(trace['trend']), 
                                                    np.median(trace['trend']) - np.percentile(trace['trend'], 16)))
    print('mu_h2: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.median(trace['trend']) + np.median(trace['delta_rv']), 
                                                    np.percentile(trace['delta_rv'], 84) - np.median(trace['delta_rv']), 
                                                    np.median(trace['delta_rv']) - np.percentile(trace['delta_rv'], 16)))
print('s_h1: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.exp(np.median(trace['logs_h1'])), 
                                                    np.exp(np.percentile(trace['logs_h1'], 84)) - np.exp(np.median(trace['logs_h1'])), 
                                                    np.exp(np.median(trace['logs_h1'])) - np.exp(np.percentile(trace['logs_h1'], 16))))
print('s_h2: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.exp(np.median(trace['logs_h2'])), 
                                                    np.exp(np.percentile(trace['logs_h2'], 84)) - np.exp(np.median(trace['logs_h2'])), 
                                                    np.exp(np.median(trace['logs_h2'])) - np.exp(np.percentile(trace['logs_h2'], 16))))

In [ ]:
plt.plot(trace['K']);

In [ ]:
varnames = ["period", "t0", "K", "trend_fwhm", "trend"]
samples = pm.trace_to_dataframe(trace, varnames=varnames)
fig = corner.corner(samples);

In [ ]:
# Get the posterior median orbital parameters
if True:
    p = np.median(trace["period"])
    t0 = np.median(trace["t0"])
    bkg_h1 = np.median(trace["bkg_h1"], axis=0)
    bkg_h2 = np.median(trace["bkg_h2"], axis=0)
    rverr_h1_corrected = np.sqrt(rverr_h1**2+np.exp(2*np.median(trace["logs_h1"])))
    rverr_h2_corrected = np.sqrt(rverr_h2**2+np.exp(2*np.median(trace["logs_h2"])))

# or get the MAP parameters instead
if False:
    p = map_soln["period"]
    t0 = map_soln["t0"]
    bkg_h1 = map_soln["bkg_h1"]
    bkg_h2 = map_soln["bkg_h2"]
    rverr_h1_corrected = np.sqrt(rverr_h1**2+np.exp(2*map_soln["logs_h1"]))
    rverr_h2_corrected = np.sqrt(rverr_h2**2+np.exp(2*map_soln["logs_h2"]))

# Plot the folded data
t_h1_fold = (t_h1 - t0 + 0.5*p) % p - 0.5*p
t_h2_fold = (t_h2 - t0 + 0.5*p) % p - 0.5*p
plt.errorbar(t_h1_fold, rv_h1 - bkg_h1, yerr=rverr_h1_corrected, fmt=",k", alpha=0.3)
plt.errorbar(t_h2_fold, rv_h2 - bkg_h2, yerr=rverr_h2_corrected, fmt=",k", alpha=0.3)
plt.errorbar(t_h1_fold, rv_h1 - bkg_h1, yerr=rverr_h1, fmt=".k", label="data")
plt.errorbar(t_h2_fold, rv_h2 - bkg_h2, yerr=rverr_h2, fmt=".k")


bins = np.linspace(-0.5 * p, 0.5*p, 7)
t_all_fold = np.concatenate([t_h1_fold, t_h2_fold])
weights_all = np.concatenate([(rv_h1 - bkg_h1) / rverr_h1_corrected**2, 
                         (rv_h2 - bkg_h2) / rverr_h2_corrected**2])
weights2_all = np.concatenate([1 / rverr_h1_corrected**2, 1 / rverr_h2_corrected**2])
num, _ = np.histogram(t_all_fold, bins, weights=weights_all)
denom, _ = np.histogram(t_all_fold, bins, weights=weights2_all)
plt.errorbar(0.5*(bins[1:]+bins[:-1]), num / denom, yerr=1 / np.sqrt(denom),
             fmt="o", color="C2", label="binned")

# Compute the posterior prediction for the folded RV model for this
# planet
t_fold = (t_dense - t0 + 0.5*p) % p - 0.5*p
inds = np.argsort(t_fold)
pred = np.percentile(trace["vrad_pred"][:, inds], [16, 50, 84], axis=0)
plt.plot(t_fold[inds], pred[1], color="C1", label="model")
art = plt.fill_between(t_fold[inds], pred[0], pred[2], color="C1", alpha=0.3)
art.set_edgecolor("none")


plt.legend(fontsize=10)
plt.xlim(-0.5*p, 0.5*p)
plt.xlabel("Phase (days)")
plt.ylabel("Radial Velocity (m s$^{-1}$)")
plt.tight_layout()
plt.savefig(plot_dir+'rvphased_fwhm_noecc_o{0}.pdf'.format(rv_trend_order));

In [ ]:
# from Tianjun:
p = 17.47129
t0 = 2458661.06279
bkg_h1 = np.zeros_like(t_h1) + 0.64
bkg_h2 = np.zeros_like(t_h2) + 1.28
rverr_h1_corrected = np.sqrt(rverr_h1**2+5.32**2)
rverr_h2_corrected = np.sqrt(rverr_h2**2+5.86**2)

# Plot the folded data
t_h1_fold = (t_h1 - t0 + 0.5*p) % p - 0.5*p
t_h2_fold = (t_h2 - t0 + 0.5*p) % p - 0.5*p
plt.errorbar(t_h1_fold, rv_h1 - bkg_h1, yerr=rverr_h1_corrected, fmt=",k", alpha=0.3)
plt.errorbar(t_h2_fold, rv_h2 - bkg_h2, yerr=rverr_h2_corrected, fmt=",k", alpha=0.3)
plt.errorbar(t_h1_fold, rv_h1 - bkg_h1, yerr=rverr_h1, fmt=".k", label="data")
plt.errorbar(t_h2_fold, rv_h2 - bkg_h2, yerr=rverr_h2, fmt=".k")


bins = np.linspace(-0.5 * p, 0.5*p, 7)
t_all_fold = np.concatenate([t_h1_fold, t_h2_fold])
weights_all = np.concatenate([(rv_h1 - bkg_h1) / rverr_h1_corrected**2, 
                         (rv_h2 - bkg_h2) / rverr_h2_corrected**2])
weights2_all = np.concatenate([1 / rverr_h1_corrected**2, 1 / rverr_h2_corrected**2])
num, _ = np.histogram(t_all_fold, bins, weights=weights_all)
denom, _ = np.histogram(t_all_fold, bins, weights=weights2_all)
plt.errorbar(0.5*(bins[1:]+bins[:-1]), num / denom, yerr=1 / np.sqrt(denom),
             fmt="o", color="C2", label="binned")


plt.legend(fontsize=10)
plt.xlim(-0.5*p, 0.5*p)
plt.xlabel("Phase (days)")
plt.ylabel("Radial Velocity (m s$^{-1}$)")
plt.tight_layout()
#plt.savefig(plot_dir+'rvphased_fwhm_noecc.pdf');

#### variable eccentricity:

In [ ]:
with pm.Model() as model:
    # Keplerian parameters
    period = pm.Normal("period", mu=17.47128, sd=0.00005)
    t0 = pm.Normal("t0", mu=2458661.0628, sd=0.0007)   
    K = pm.Uniform("K", lower=0., upper=10., testval=5.)
    ecc = xo.distributions.eccentricity.kipping13("ecc", testval=0.01)
    omega = xo.distributions.Angle("omega", testval=np.pi)
    
    # RV offsets
    delta_rv = pm.Uniform("delta_rv", lower=-10., upper=10.) # mu_h2 - trend[1]
    if rv_trend_order == 1:
        trend = pm.Uniform("trend", lower=-10., upper=10.) # ideally this would be shape [1] but idk how to make that work so terrible hacks will follow
        #trend = pm.Normal("trend", mu=0, sd=3)
    else:
        trend = pm.Normal("trend", mu=0, sd=10.0**(1-np.arange(rv_trend_order))[::-1], shape=rv_trend_order)
    
    # RV jitter
    logs_h1 = pm.Uniform("logs_h1", lower=0., upper=10.)
    logs_h2 = pm.Uniform("logs_h2", lower=0., upper=10.)
    
    # FWHM trend
    delta_fwhm = pm.Normal("delta_fwhm", mu=1., sd=5) # weak prior from eyeballing
    trend_fwhm = pm.Normal("trend_fwhm", mu=0, sd=10.0**(1-np.arange(2))[::-1], shape=2)
            
    # Orbit model
    orbit_rvs = xo.orbits.KeplerianOrbit(
        period=period,
        ecc=ecc, omega=omega)
    
    # Save some helpful things for later
    semimajor = orbit_rvs.a
    pm.Deterministic('a', semimajor)
    
    # Set up the RV model and save it as a deterministic
    # for plotting purposes later
    vrad_h1 = orbit_rvs.get_radial_velocity(t_h1, K=K)
    vrad_h2 = orbit_rvs.get_radial_velocity(t_h2, K=K)
    vrad = orbit_rvs.get_radial_velocity(np.append(t_h1, t_h2), K=K)
    pm.Deterministic("vrad", vrad)
    
    # RV noise model for h1
    A_fwhm_h1 = np.vander(fwhm_h1, 2)
    bkg_terms_h1 = tt.dot(A_fwhm_h1, trend_fwhm)
    if rv_trend_order == 1:
        bkg_terms_h1 += trend
    else:
        A_trend_h1 = np.vander(t_h1, rv_trend_order)
        bkg_terms_h1 += tt.dot(A_trend_h1, trend)
    bkg_h1 = pm.Deterministic("bkg_h1", bkg_terms_h1)
        
    # RV noise model for h2
    A_fwhm_h2 = np.vander(fwhm_h2, 2)
    bkg_terms_h2 = tt.dot(A_fwhm_h2, trend_fwhm) - delta_fwhm*trend_fwhm[0]
    if rv_trend_order == 1:
        bkg_terms_h2 += trend + delta_rv
    else:
        A_trend_h2 = np.vander(t_h2, rv_trend_order)
        bkg_terms_h2 += tt.dot(A_trend_h2, trend) + delta_rv
    bkg_h2 = pm.Deterministic("bkg_h2", bkg_terms_h2)

    # The likelihood for the RVs
    rv_model_h1 = pm.Deterministic("rv_model_h1", vrad_h1 + bkg_h1)  
    rv_model_h2 = pm.Deterministic("rv_model_h2", vrad_h2 + bkg_h2)        
    err_h1 = tt.sqrt(rverr_h1**2 + tt.exp(2*logs_h1))
    err_h2 = tt.sqrt(rverr_h2**2 + tt.exp(2*logs_h2))
    pm.Normal("obs_h1", mu=rv_model_h1, sd=err_h1, observed=rv_h1)
    pm.Normal("obs_h2", mu=rv_model_h2, sd=err_h2, observed=rv_h2)
        
    vrad_pred = orbit_rvs.get_radial_velocity(t_dense, K=K)
    pm.Deterministic("vrad_pred", vrad_pred)
    A_pred = np.vander(t_dense, rv_trend_order)        
    bkg_pred = pm.Deterministic("bkg_pred", tt.dot(A_pred, trend))
    pm.Deterministic("rv_model_pred", vrad_pred + bkg_pred)

    # Fit for the maximum a posteriori parameters
    start = model.test_point
    map_soln = start
    map_soln = xo.optimize(start=map_soln, vars=[logs_h1, logs_h2, trend, delta_rv])
    map_soln = xo.optimize(start=map_soln, vars=[trend_fwhm, delta_fwhm])
    map_soln = xo.optimize(start=map_soln, vars=[period, t0])
    map_soln = xo.optimize(start=map_soln, vars=[K])
    map_soln = xo.optimize(start=map_soln, vars=[ecc, omega])
    map_soln = xo.optimize(start=map_soln)

In [ ]:
if run_trace:
    with model:
        trace = pm.sample(tune=1000, draws=1000, start=map_soln, chains=2,
                      step=xo.get_dense_nuts_step(target_accept=0.9))
    pm.save_trace(trace,directory='trace_fwhm_o{0}'.format(rv_trend_order), overwrite=True)   
else:
    with model:
        trace = pm.load_trace('trace_fwhm_o{0}'.format(rv_trend_order))

In [ ]:
for varname in ["period", "t0", "K", "delta_rv", "delta_fwhm", 
                "trend", "trend_fwhm", "logs_h1", "logs_h2",
                "ecc", "omega"]:
    print("{0}: median {1} +{2} -{3}; MAP {4}".format(varname, np.median(trace[varname], axis=0), 
                                                      np.percentile(trace[varname], [84], axis=0) - np.median(trace[varname], axis=0), 
                                                      np.median(trace[varname], axis=0) - np.percentile(trace[varname], [16], axis=0), 
                                                      map_soln[varname]))

In [ ]:
print('period: {0:.7f} + {1:.7f} - {2:.7f}'.format(np.median(trace['period']), 
                                                    np.percentile(trace['period'], 84) - np.median(trace['period']), 
                                                    np.median(trace['period']) - np.percentile(trace['period'], 16)))
print('t0: {0:.5f} + {1:.5f} - {2:.5f}'.format(np.median(trace['t0']), 
                                                    np.percentile(trace['t0'], 84) - np.median(trace['t0']), 
                                                    np.median(trace['t0']) - np.percentile(trace['t0'], 16)))
print('K: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.median(trace['K']), 
                                                    np.percentile(trace['K'], 84) - np.median(trace['K']), 
                                                    np.median(trace['K']) - np.percentile(trace['K'], 16)))
if rv_trend_order>1:
    print('mu_h1: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.median(trace['trend'], axis=0)[-1], 
                                                    np.percentile(trace['trend'], 84, axis=0)[-1] - np.median(trace['trend'], axis=0)[-1], 
                                                    np.median(trace['trend'], axis=0)[-1] - np.percentile(trace['trend'], 16, axis=0)[-1]))
    print('mu_h2: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.median(trace['trend'], axis=0)[-1] + np.median(trace['delta_rv']), 
                                                    np.percentile(trace['delta_rv'], 84) - np.median(trace['delta_rv']), 
                                                    np.median(trace['delta_rv']) - np.percentile(trace['delta_rv'], 16)))
else:
    print('mu_h1: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.median(trace['trend']), 
                                                    np.percentile(trace['trend'], 84) - np.median(trace['trend']), 
                                                    np.median(trace['trend']) - np.percentile(trace['trend'], 16)))
    print('mu_h2: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.median(trace['trend']) + np.median(trace['delta_rv']), 
                                                    np.percentile(trace['delta_rv'], 84) - np.median(trace['delta_rv']), 
                                                    np.median(trace['delta_rv']) - np.percentile(trace['delta_rv'], 16)))
print('s_h1: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.exp(np.median(trace['logs_h1'])), 
                                                    np.exp(np.percentile(trace['logs_h1'], 84)) - np.exp(np.median(trace['logs_h1'])), 
                                                    np.exp(np.median(trace['logs_h1'])) - np.exp(np.percentile(trace['logs_h1'], 16))))
print('s_h2: {0:.2f} + {1:.2f} - {2:.2f}'.format(np.exp(np.median(trace['logs_h2'])), 
                                                    np.exp(np.percentile(trace['logs_h2'], 84)) - np.exp(np.median(trace['logs_h2'])), 
                                                    np.exp(np.median(trace['logs_h2'])) - np.exp(np.percentile(trace['logs_h2'], 16))))

In [ ]:
varnames = ["period", "t0", "K", "ecc", "omega", "trend_fwhm", "trend"]
samples = pm.trace_to_dataframe(trace, varnames=varnames)
fig = corner.corner(samples);

In [ ]:
# Get the posterior median orbital parameters
if True:
    p = np.median(trace["period"])
    t0 = np.median(trace["t0"])
    bkg_h1 = np.median(trace["bkg_h1"], axis=0)
    bkg_h2 = np.median(trace["bkg_h2"], axis=0)
    rverr_h1_corrected = np.sqrt(rverr_h1**2+np.exp(2*np.median(trace["logs_h1"])))
    rverr_h2_corrected = np.sqrt(rverr_h2**2+np.exp(2*np.median(trace["logs_h2"])))

# or get the MAP parameters instead
if False:
    p = map_soln["period"]
    t0 = map_soln["t0"]
    bkg_h1 = map_soln["bkg_h1"]
    bkg_h2 = map_soln["bkg_h2"]
    rverr_h1_corrected = np.sqrt(rverr_h1**2+np.exp(2*map_soln["logs_h1"]))
    rverr_h2_corrected = np.sqrt(rverr_h2**2+np.exp(2*map_soln["logs_h2"]))

# Plot the folded data
t_h1_fold = (t_h1 - t0 + 0.5*p) % p - 0.5*p
t_h2_fold = (t_h2 - t0 + 0.5*p) % p - 0.5*p
plt.errorbar(t_h1_fold, rv_h1 - bkg_h1, yerr=rverr_h1_corrected, fmt=",k", alpha=0.3)
plt.errorbar(t_h2_fold, rv_h2 - bkg_h2, yerr=rverr_h2_corrected, fmt=",k", alpha=0.3)
plt.errorbar(t_h1_fold, rv_h1 - bkg_h1, yerr=rverr_h1, fmt=".k", label="data")
plt.errorbar(t_h2_fold, rv_h2 - bkg_h2, yerr=rverr_h2, fmt=".k")


bins = np.linspace(-0.5 * p, 0.5*p, 7)
t_all_fold = np.concatenate([t_h1_fold, t_h2_fold])
weights_all = np.concatenate([(rv_h1 - bkg_h1) / rverr_h1_corrected**2, 
                         (rv_h2 - bkg_h2) / rverr_h2_corrected**2])
weights2_all = np.concatenate([1 / rverr_h1_corrected**2, 1 / rverr_h2_corrected**2])
num, _ = np.histogram(t_all_fold, bins, weights=weights_all)
denom, _ = np.histogram(t_all_fold, bins, weights=weights2_all)
plt.errorbar(0.5*(bins[1:]+bins[:-1]), num / denom, yerr=1 / np.sqrt(denom),
             fmt="o", color="C2", label="binned")

# Compute the posterior prediction for the folded RV model for this
# planet
t_fold = (t_dense - t0 + 0.5*p) % p - 0.5*p
inds = np.argsort(t_fold)
pred = np.percentile(trace["vrad_pred"][:, inds], [16, 50, 84], axis=0)
plt.plot(t_fold[inds], pred[1], color="C1", label="model")
art = plt.fill_between(t_fold[inds], pred[0], pred[2], color="C1", alpha=0.3)
art.set_edgecolor("none")


plt.legend(fontsize=10)
plt.xlim(-0.5*p, 0.5*p)
plt.xlabel("Phase (days)")
plt.ylabel("Radial Velocity (m s$^{-1}$)")
plt.tight_layout()
plt.savefig(plot_dir+'rvphased_fwhm_o{0}.pdf'.format(rv_trend_order));